In [6]:
# Bloque 1: Instalación de dependencias y verificación de GPU
!pip install ultralytics numpy pandas matplotlib seaborn
!nvidia-smi  # Verificar GPU disponible

# Bloque 2: Importaciones y configuración inicial
from google.colab import drive
import os
from ultralytics import YOLO
import torch
import yaml
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import json
from datetime import datetime

# Bloque 3: Montar Drive y configurar rutas
drive.mount('/content/drive', force_remount=True)
time.sleep(2)  # Esperar a que Drive se monte correctamente

# Configurar rutas
BASE_PATH = '/content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2'
DATASET_PATH = f'{BASE_PATH}/roboflow_dataset'
MODEL_PATH = f'{BASE_PATH}/models'
RESULTS_PATH = f'{BASE_PATH}/results'

# Crear directorios necesarios
os.makedirs(RESULTS_PATH, exist_ok=True)

# Verificar GPU
print(f"\nDispositivo disponible: {'cuda' if torch.cuda.is_available() else 'cpu'}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")



Fri Jan 17 13:36:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
# Bloque 4: Funciones auxiliares (mantener las existentes y agregar nuevas)
def save_metrics_to_json(metrics, filename):
    """Guarda las métricas en un archivo JSON"""
    metrics_dict = {k: v[0] if isinstance(v, list) else v for k, v in metrics.items()}
    filepath = os.path.join(RESULTS_PATH, filename)
    with open(filepath, 'w') as f:
        json.dump(metrics_dict, f, indent=4)
    print(f"\n✅ Métricas guardadas en: {filepath}")

def save_training_config(config):
    """Guarda la configuración del entrenamiento"""
    filepath = os.path.join(RESULTS_PATH, 'training_config.json')
    with open(filepath, 'w') as f:
        json.dump(config, f, indent=4)
    print(f"\n✅ Configuración guardada en: {filepath}")



In [8]:
# Bloque 5: Función de entrenamiento optimizada
def train_model(epochs=50, patience=10, batch_size=16):
    """
    Entrena el modelo YOLO con early stopping y guardado del mejor modelo
    """
    print("\nVerificando dataset antes del entrenamiento...")
    if not verify_dataset():
        print("❌ Entrenamiento cancelado: Problemas con el dataset")
        return None

    print("\n🚀 Iniciando entrenamiento...")
    try:
        # Configuración del entrenamiento
        config = {
            'epochs': epochs,
            'batch_size': batch_size,
            'image_size': 640,
            'device': 'cuda' if torch.cuda.is_available() else 'cpu',
            'model_type': 'yolov8n.pt',
            'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S")
        }

        # Guardar configuración
        save_training_config(config)

        # Inicializar modelo
        model = YOLO('yolov8n.pt')

        # Configurar entrenamiento
        results = model.train(
            data=f'{DATASET_PATH}/data.yaml',
            epochs=epochs,
            imgsz=config['image_size'],
            batch=batch_size,
            device=config['device'],
            project=MODEL_PATH,
            name=f'logo_detection_{config["timestamp"]}',
            patience=patience,  # Early stopping
            save=True,  # Guardar mejor modelo
            save_period=10,  # Guardar cada 10 epochs
            plots=True  # Generar gráficos de entrenamiento
        )

        print("✅ Entrenamiento completado exitosamente")
        return results

    except Exception as e:
        print(f"❌ Error durante el entrenamiento: {str(e)}")
        return None



In [9]:
# Bloque 6: Función de evaluación mejorada
def evaluate_model(model_path, val_data_path):
    """
    Evalúa un modelo entrenado y guarda resultados
    """
    try:
        # Cargar modelo
        model = YOLO(model_path)
        print(f"\n📊 Evaluando modelo: {os.path.basename(model_path)}")

        # Calcular métricas
        metrics = calculate_metrics(model, val_data_path)

        # Imprimir resumen
        print_metrics_summary(metrics)

        # Generar y guardar visualizaciones
        fig = plot_metrics(metrics)
        metrics_plot_path = os.path.join(RESULTS_PATH, 'evaluation_metrics.png')
        fig.savefig(metrics_plot_path)
        plt.close(fig)

        # Guardar métricas en JSON
        save_metrics_to_json(metrics, 'model_metrics.json')

        return metrics

    except Exception as e:
        print(f"❌ Error durante la evaluación: {str(e)}")
        return None



In [10]:
# Bloque 7: Función principal de ejecución
def run_training_and_evaluation(epochs=50, patience=10, batch_size=16):
    """
    Ejecuta el proceso completo de entrenamiento y evaluación
    """
    print("\n🔄 Iniciando proceso completo...")

    # Entrenamiento
    results = train_model(epochs=epochs, patience=patience, batch_size=batch_size)

    if results is None:
        return

    # Obtener ruta del mejor modelo
    best_model_path = results.best

    # Evaluación
    metrics = evaluate_model(best_model_path, f'{DATASET_PATH}/data.yaml')

    if metrics is not None:
        print(f"\n✨ Proceso completado exitosamente")
        print(f"📁 Mejor modelo guardado en: {best_model_path}")
        print(f"📊 Resultados guardados en: {RESULTS_PATH}")

# Ejemplo de uso:
# run_training_and_evaluation(epochs=50, patience=10, batch_size=16)

In [11]:
# Bloque 8: Bloque principal de ejecución
if __name__ == "__main__":
    print("\n=== 🚀 Iniciando Pipeline de Entrenamiento y Evaluación ===")

    # Parámetros configurables
    PARAMS = {
        'epochs': 50,        # Número de epochs para entrenamiento
        'patience': 10,      # Epochs sin mejora antes de early stopping
        'batch_size': 16     # Tamaño del batch (ajustar según GPU)
    }

    # Verificar GPU y mostrar información
    print("\n=== 🖥️ Información del Sistema ===")
    print(f"GPU Disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"Modelo GPU: {torch.cuda.get_device_name(0)}")
        print(f"Memoria GPU Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

    # Verificar directorios y dataset
    print("\n=== 📁 Verificación de Directorios ===")
    print(f"Base Path: {BASE_PATH}")
    print(f"Dataset Path: {DATASET_PATH}")
    print(f"Model Path: {MODEL_PATH}")
    print(f"Results Path: {RESULTS_PATH}")

    # Ejecutar pipeline completo
    print("\n=== 🏃‍♂️ Ejecutando Pipeline ===")
    print(f"Epochs: {PARAMS['epochs']}")
    print(f"Early Stopping Patience: {PARAMS['patience']}")
    print(f"Batch Size: {PARAMS['batch_size']}")

    try:
        run_training_and_evaluation(
            epochs=PARAMS['epochs'],
            patience=PARAMS['patience'],
            batch_size=PARAMS['batch_size']
        )
        print("\n=== ✅ Pipeline Completado Exitosamente ===")

    except Exception as e:
        print("\n=== ❌ Error en el Pipeline ===")
        print(f"Error: {str(e)}")

    finally:
        print("\n=== 🏁 Proceso Finalizado ===")


=== 🚀 Iniciando Pipeline de Entrenamiento y Evaluación ===

=== 🖥️ Información del Sistema ===
GPU Disponible: True
Modelo GPU: Tesla T4
Memoria GPU Total: 15.84 GB

=== 📁 Verificación de Directorios ===
Base Path: /content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2
Dataset Path: /content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2/roboflow_dataset
Model Path: /content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2/models
Results Path: /content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2/results

=== 🏃‍♂️ Ejecutando Pipeline ===
Epochs: 50
Early Stopping Patience: 10
Batch Size: 16

🔄 Iniciando proceso completo...

Verificando dataset antes del entrenamiento...

Contenido de train:
📸 Imágenes encontradas: 606
🏷️ Labels encontrados: 606

Contenido de valid:
📸 Imágenes encontradas: 46
🏷️ Labels encontrados: 46

Contenido de test:
📸 Imágenes encontradas: 20
🏷️ Labels encontrados: 20

🚀 Iniciando entrenamiento...

✅ Configuraci

100%|██████████| 6.25M/6.25M [00:00<00:00, 228MB/s]

Ultralytics 8.3.63 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2/roboflow_dataset/data.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=cuda, workers=8, project=/content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2/models, name=logo_detection_20250117_133805, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fal

100%|██████████| 755k/755k [00:00<00:00, 107MB/s]


...

      26/50      2.18G      1.362     0.8535      1.359         57        640: 100%|██████████| 38/38 [00:15<00:00,  2.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.45it/s]

                   all         46          6      0.382      0.667      0.575      0.408
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 16, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



26 epochs completed in 0.121 hours.
Optimizer stripped from /content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2/models/logo_detection_20250117_133805/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2/models/logo_detection_20250117_133805/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2/models/logo_detection_20250117_133805/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.13it/s]


                   all         46          6          1      0.617      0.796      0.507
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /content/drive/MyDrive/BootcampAI/ComputerVision/logo_detection_dir2/models/logo_detection_20250117_133805
✅ Entrenamiento completado exitosamente

=== ❌ Error en el Pipeline ===
Error: 'DetMetrics' object has no attribute 'best'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP) of an
    object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (dict of str): A dict of strings